# Installations

In [40]:
pip install ipyfilechooser ipywidgets

Note: you may need to restart the kernel to use updated packages.


# Data Exploration

### T10-4
As a USER, I want to have a Data Exploration Section in the notebook so that I can easily locate the upload and playback functions in the notebook.

# Inference

### T10-7
As a USER, I want to have an Inference Section in the notebook so that I can easily locate the Inference Section in the notebook.

# Training

### T10-6
As a USER, I want to have a Training Section in the notebook so that I can easily locate the training related functions in the notebook.

In [1]:
import ipywidgets as widgets
from IPython.display import display, HTML
from ipyfilechooser import FileChooser
import zipfile as zf
from ipywidgets import Button

# ========== TOGGLE BTN ==========
javascript_functions = {False: "hide()", True: "show()"}
button_descriptions  = {False: "Show code", True: "Hide code"}

def toggle_code(state):
    """
    Toggles the JavaScript show()/hide() function on the div.input element.
    """
    output_string = "<script>$(\"div.input\").{}</script>"
    output_args   = (javascript_functions[state],)
    output        = output_string.format(*output_args)
    display(HTML(output))

def button_action(value):
    """
    Calls the toggle_code function and updates the button description.
    """
    state = value.new
    toggle_code(state)
    value.owner.description = button_descriptions[state]

state = False
toggle_code(state)

button = widgets.ToggleButton(state, description = button_descriptions[state])
button.observe(button_action, "value")
# ========== END TOGGLE BTN END ==========
# ========== FileChooser ==========
# Create and display a FileChooser widget
fc = FileChooser('/Users')

# Change defaults and reset the dialog
# fc.default_path = './'
# fc.reset()

# Restrict navigation to /Users
fc.sandbox_path = '/Users'

# Change hidden files
fc.show_hidden = False

# Switch to folder-only mode
fc.show_only_dirs = False

# Set multiple file filter patterns (uses https://docs.python.org/3/library/fnmatch.html)
# fc.filter_pattern = ['*.jpg', '*.png','*.txt']
fc.filter_pattern = ['*.zip']

# Change the title (use '' to hide)
fc.title = '<b>Select File (*zip)</b>'

# Sample callback function
def change_display_selected(chooser):
    print(fc.selected_filename, end='\r')

# Register callback function
fc.register_callback(change_display_selected)

def on_button_clicked(b):
    # FILE UPLOAD
    print(fc.selected)
    fn = fc.selected_filename.replace('.zip', '')
    files = zf.ZipFile(fc.selected_path+"\\"+fc.selected_filename, 'r')
    files.extractall('Data Folder\\'+fn)
    files.close()
    with output:
        print("Files saved.")
# ========== END FileChooser END ==========
# ========== CONFIRM BTN ==========
confirmBtn = widgets.Button(
    description='Confirm Upload',
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Confirm',
)
output = widgets.Output()
def on_button_clicked(b):
    # FILE UPLOAD
    print(fc.selected)
    fn = fc.selected_filename.replace('.zip', '')
    files = zf.ZipFile(fc.selected_path+"\\"+fc.selected_filename, 'r')
    files.extractall('Data Folder\\'+fn)
    files.close()
    with output:
        print("Files saved.")

confirmBtn.on_click(on_button_clicked)
# ========== CONFIRM BTN ==========

display(button)
display(fc)
display(confirmBtn, output)

ToggleButton(value=False, description='Show code')

FileChooser(path='C:\Users', filename='', title='<b>Select File (*zip)</b>', show_hidden=False, select_desc='S…

Button(button_style='info', description='Confirm Upload', style=ButtonStyle(), tooltip='Confirm')

Output()

C:\Users\jaspz\Documents\3104 - Software Management\pfile.zip
